<a href="https://colab.research.google.com/github/AntJuLRa/ANLP-project-/blob/dev/generation/n-gram_markov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk import FreqDist
from nltk import ngrams
import itertools
import re
import random
import dill

In [ ]:
filename = 'n-gram_markov.pkl'

In [ ]:
dill.dump_session(filename)

In [ ]:
dill.load_session(filename)

**some dummy data preprocessing**

In [ ]:
data = pd.read_csv('output.txt', sep='\n\n\n', engine='python',encoding='utf8', header = 0)

In [ ]:
data.head()

,Artist: 40 Thevz f/ Malika
0,Album: Honor Amongst Thevz
1,Song: All I Wanna Do
2,"Chorus: Malika, P.S."
3,All I wanna do is spend some time with you.
4,And all I wanna do is bump 'n grind with you.


In [ ]:
linelist= data["Artist: 40 Thevz f/ Malika"]

In [ ]:
lines = [x for x in linelist]

In [ ]:
lines = [x for x in lines if x[:5] != 'Album' and x[:6]!='Artist' and x[0]!='*' and x[:5]!='Typed']

In [ ]:
def replace_verse(x):
    if x[:5]=='Verse' or x[:6]=='Chorus' or x[:5]=='Intro' or x[0]=='[':
        return "NXTVRSE"
    if x[:4]=='Song':
        return "NXTSNG"
    else: 
        return x

In [ ]:
lines = [replace_verse(x) for x in lines]

In [ ]:
mini_lines = [re.sub("[^A-Za-z0-9\-\ ]", "", x.lower()) for x in lines]

In [ ]:
from nltk.tokenize.sonority_sequencing import SyllableTokenizer
from nltk.tokenize import TreebankWordTokenizer

In [ ]:
word_lines = [TreebankWordTokenizer().tokenize(x) for x in mini_lines]

In [ ]:
syllable_lines = [[SyllableTokenizer().tokenize(x) for x in y] for y in word_lines]

In [ ]:
letter_lines = [list(x) for x in mini_lines]

In [ ]:
all_lines = [x for x in word_lines if len(x)>0]
for y in all_lines:
    if y[0]!='nxtsng' and y[0]!='nxtvrse':
        y.append('endline')

In [ ]:
dummy = all_lines

In [ ]:
long_dummy = [x for y in dummy for x in y]

In [ ]:
size = len(long_dummy) 
idx_list = [idx + 1 for idx, val in enumerate(long_dummy) if val == 'nxtsng'] 
  
  
listlist = [long_dummy[i: j] for i, j in
        zip([0] + idx_list, idx_list + 
        ([size] if idx_list[-1] != size else []))] 

**Plain n-gram markov model for generation**

In [ ]:
class ngram_markov_generator(object):
    
    def __init__(self, order,  end='nxtsng', endline='endline', meta_list=['nxtvrse','nxtsng']):
        self.end = end
        self.endline = endline
        self.meta_list = meta_list
        self.order =order
        self.freq_dict = dict()
        
    def train(self, tknzd_txt_list):
        for text in tknzd_txt_list:
            grams = list(ngrams(text, self.order+1))
            for gram in grams:
                self.add_to_dict(gram)
            
    def add_to_dict(self, gram):
        try:
            self.freq_dict[gram[:-1]][gram[-1]]+=1
        except KeyError:
            self.freq_dict[gram[:-1]]= FreqDist([gram[-1]])
    
    def generate_text(self, start, max_len=20, temp=1):
        key = start[-self.order:]
        res_sent= start
        
        for _ in itertools.repeat(None, max_len):
            
            with_temp = {key: value**(1/temp) for key, value in self.freq_dict[tuple(key)].items()}
            dist = nltk.DictionaryProbDist(with_temp,normalize=True)
            
            nextword = dist.generate()
            res_sent.append(nextword)
            
            if nextword==self.end:
                break
                
            key =res_sent[-self.order:]
        
        return res_sent
    
    def generate_lines(self, start, num_lines, max_len=200, temp=1):
        key = start[-self.order:]
        res_sent= start
        linecount=0
        for _ in itertools.repeat(None, max_len):
            
            with_temp = {key: value**(1/temp) for key, value in self.freq_dict[tuple(key)].items()}
            dist = nltk.DictionaryProbDist(with_temp,normalize=True)
            
            nextword = dist.generate()
            res_sent.append(nextword)
            
            if nextword==self.endline:
                linecount = linecount + 1
            
            if linecount >= num_lines:
                break
            if nextword==self.end:
                break
                
            key =res_sent[-self.order:]
        
        return res_sent

    def nice_format(self, output_list):
        no_meta = [x for x in output_list if x not in self.meta_list]
        with_linebreaks = ["\n" if x==self.endline else x for x in no_meta]
        return " ".join(with_linebreaks)      

In [ ]:
trigram_mc = ngram_markov_generator(2)

In [ ]:
trigram_mc.train(listlist)

In [ ]:
text= trigram_mc.generate_text(['nxtvrse','i','was','so'],max_len=20,temp=1)
print(trigram_mc.nice_format(text))

i was so regular 
 i appeared like a witch trapped in lying 
 while youre looking through the first time show niggas


In [ ]:
lines= trigram_mc.generate_lines(['nxtvrse','i','was','so'],4)
print(trigram_mc.nice_format(lines))

i was so beautiful from your front lawn 
 everywhere we makin ya dance every dance got ta have it on your ass was to be in all of you bitches so vicious im still living 
 who im not the past that comes from 
 that you fell the pressure and pain take it from my golf swing 



**adding constraints**